## Test functions used for benchmarking

Every test function is a method that takes as inputs the number of dimensions, the domain and further parameters. The output is the function value at $ x $. Further the optimal solution is stored.
- Divide problems into easy and hard
- Create starting points separately
- 

In [2]:
# Load some libraries first
import numpy as np

### Single objective optimization

In [7]:
class Rastrigin:
    
    def __init__(self, dim):
        self.dim = dim
        self.solution = np.zeros(dim)
        # Define lower and upper bounds of domain
        self.lb = np.repeat(-5.12, dim)
        self.ub = np.repeat(+5.12, dim)
        
    def fun(self, x):
        A = 10
    
        if np.any(np.abs(x) > 5.12):
            raise ValueError("The absolute value of x exceeded 5.12")
        
        return A * self.dim + np.sum(x ** 2 - A * np.cos(2 * np.pi * x))

class Ackley:
    
    def __init__(self):
        self.solution = np.zeros(2)
        # Define lower and upper bounds of domain
        self.lb = [-5, -np.inf]
        self.ub = [np.inf, 5]

    def fun(self, x):
        if (x[0] < -5) | (x[1] > 5):
            raise ValueError("It must hold that x[0] >= -5 and x[1] <= 5.")

        return (-20 * np.exp(-0.2 * np.sqrt(0.5 * (x[0] ** 2 + x[1] ** 2)))
               - np.exp(0.5 * (np.cos(2 * np.pi * x[0]) + np.cos(2 * np.pi * x[1])))
               + np.exp(1) + 20)

class Sphere:
    
    def __init__(self, dim):
        self.solution = np.zeros(dim)
        # Define lower and upper bounds of domain
        self.lb = np.repeat(-np.inf, dim)
        self.ub = np.repeat(np.inf, dim)

    def fun(self, x):
        return np.sum(x ** 2)


class Rosenbrock:

    def __init__(self, dim):
        self.solution = np.ones(dim)
        # Define lower and upper bounds of domain
        self.lb = np.repeat(-np.inf, dim)
        self.ub = np.repeat(np.inf, dim)
    
    def fun(self, x):
        return np.sum(100 * (x[1::] - x[0:-1] ** 2) ** 2 + (1 - x[0:-1]) ** 2)

class Beale:

    def __init__(self):
        self.solution = [3, 0.5]
        # Define lower and upper bounds of domain
        self.lb = [-4.5, -np.inf]
        self.ub = [np.inf, 4.5]
    
    def fun(self, x):
        if (x[0] < -4.5) | (x[1] > 4.5):
            raise ValueError("It must hold that x[0] >= -4.5 and x[1] <= 4.5.")
        
        return ((1.5 - x[0] + x[0] * x[1]) ** 2 + (2.25 - x[0] + x[0] * x[1] ** 2) ** 2 
               + (2.625 - x[0] + x[0] * x[1] ** 3))

class Himmelblau:
    
    def __init__(self):
        self.solution = np.array(
            [3, 2, -2.805118, 3.131312, -3.779310, -3.283186, 3.584428, -1.848126])
        # Define lower and upper bounds of domain
        self.lb = [-5, -np.inf]
        self.ub = [np.inf, 5]
    
    def fun(self, x):
        if (x[0] < -5) | (x[1] > 5):
            raise ValueError("It must hold that x >= -5 and y <= 5.")
            
        return (x[0] ** 2 + x[1] - 11) ** 2 + (x[0] + x[1] ** 2 - 7) ** 2


class McCormick:
    
    def __init__(self):
        self.solution = [-0.54719, -1.54719]
        # Define lower and upper bounds of domain
        self.lb = [-1.5, -3]
        self.ub = [4, 4]
    
    def fun(self, x):
        if (x[0] < -1.5) | (x[0] > 4) | (x[1] < -3) | (x[1] > 4):
            raise ValueError("Domain missspecified x[0] in [-1.5, 4] and x[1] in [-3, 4]")

        return (np.sin(x[0] + x[1]) + (x[0] - x[1]) ** 2 - 1.5 * x[0] + 2.5 * x[1] + 1)

class Schaffer_No2:
    
    def __init__(self):
        self.solution = np.zeros(2)
        # Define lower and upper bounds of domain
        self.lb = [-100, -np.inf]
        self.ub = [np.inf, 100]
    
    def fun(self, x):
        if (x[0] < -100) | (x[1] > 100):
            raise ValueError("It must hold that x >= -100 and y <= 100")

        return (0.5 + (np.sin(x[0] ** 2 - x[1] ** 2) ** 2 - 0.5) 
               / (1 + 0.001 * (x[0] ** 2 + x[1] ** 2)) ** 2)

### Constrained Optimization

### Multiobjective Function

### Economic Functions 
- Dynamic Programming
- Some market formulas

### Showcase

In [6]:
ackley = Ackley()
ackley.fun([1, 0])

2.637531092108304